# Combine SHMI data for a parameter

In [11]:
import os
import pandas as pd

def combine_and_sum_csv(folder_path, column_index):
    
    csv_files = [f for f in os.listdir(folder_path) if f.endswith(".csv")]
    
    dfs = []

    for file in csv_files:
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        
        df["Datum"] = pd.to_datetime(df["Datum"], format='%Y-%m-%d')
    
        df = df[df["Datum"].dt.year >= 2019]
        
        dfs.append(df)
    combined_df = pd.concat(dfs, ignore_index=True)

    if column_index >= len(combined_df.columns):
        print(f"Invalid column index {column_index}. There are only {len(combined_df.columns)} columns.")
        return
    
    column_name = combined_df.columns[column_index]

    grouped_df = combined_df.groupby('Datum')[column_name].sum()

    folder_name = os.path.basename(folder_path)
    output_file_name = f"consolidated_{folder_name}.csv"

    output_file = os.path.join("smhi_data_2022-today", output_file_name)

    grouped_df.to_csv(output_file)

    print(f"Saved to: {output_file}")


In [12]:
combine_and_sum_csv(r"smhi_data_2022-today/parameter_2", 1)

Saved to: smhi_data_2022-today\consolidated_parameter_2.csv


In [13]:
combine_and_sum_csv(r"smhi_data_2022-today/parameter_5", 1)

Saved to: smhi_data_2022-today\consolidated_parameter_5.csv


In [14]:
combine_and_sum_csv(r"smhi_data_2022-today/parameter_8", 2)

Saved to: smhi_data_2022-today\consolidated_parameter_8.csv


C:\Users\sneha\AppData\Local\Temp\ipykernel_15444\3476833433.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat(dfs, ignore_index=True)


In [17]:
import os
import pandas as pd

def combine_and_sum_solskenstid_from_2019(folder_path):
    csv_files = [f for f in os.listdir(folder_path) if f.endswith(".csv")]
    dfs = []

    for file in csv_files:
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        
        df["Datum"] = pd.to_datetime(df["Datum"], errors='coerce')
        
        df = df[df["Datum"].dt.year >= 2019]
        
        df["Tid (UTC)"] = pd.to_datetime(df["Tid (UTC)"], format='%H:%M:%S').dt.strftime('%H:%M:%S')
        
        dfs.append(df)
    combined_df = pd.concat(dfs, ignore_index=True)

    grouped_df = combined_df.groupby(['Datum', 'Tid (UTC)'])['Solskenstid'].sum().reset_index()

    folder_name = os.path.basename(folder_path)
    output_file_name = f"consolidated_{folder_name}.csv"
    output_file = os.path.join("smhi_data_2022-today", output_file_name)

    grouped_df.to_csv(output_file, index=False)

    print(f"Saved to: {output_file}")




In [18]:
combine_and_sum_solskenstid_from_2019(r"smhi_data_2022-today/parameter_10")

Saved to: smhi_data_2022-today\consolidated_parameter_10.csv


# Converting day values to hourly values

In [89]:
import os
import pandas as pd
from datetime import datetime, timedelta

def convert_daily_to_hourly(folder_path, file_name, column_index):

    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path)

    df["Datum"] = pd.to_datetime(df["Datum"], errors="coerce")

    invalid_dates = df[df["Datum"].isna()]

    df = df.dropna(subset=["Datum"])

    if "parameter_2" in file_name.lower():
        interpolated_column_name = "Air temperature"
    elif "parameter_5" in file_name.lower():
        interpolated_column_name = "Precipitation"
    elif "parameter_8" in file_name.lower():
        interpolated_column_name = "Snow depth"
    else:
        interpolated_column_name = "Interpolated Value"

    hourly_df = []

    for _, row in df.iterrows():
        day = row["Datum"]
        value = row.iloc[column_index] 

        for hour in range(24):
            hourly_time = day + timedelta(hours=hour)

            datum = hourly_time.strftime('%d-%m-%Y') 
            tid_utc = hourly_time.strftime('%H:%M:%S')

            hourly_df.append({
                "Datum": datum,
                "Tid (UTC)": tid_utc,
                interpolated_column_name: value
            })

    hourly_df = pd.DataFrame(hourly_df)

    # Save the hourly data to a new CSV file
    output_file_name = f"hourly_{file_name}"
    output_file_path = os.path.join(folder_path, output_file_name)
    hourly_df.to_csv(output_file_path, index=False)

    print(f"Hourly data saved to: {output_file_path}")


In [90]:
convert_daily_to_hourly(r"smhi_data_2022-today", "consolidated_parameter_2.csv", 1)

Raw 'Datum' values:
0    2019-01-01
1    2019-01-02
2    2019-01-03
3    2019-01-04
4    2019-01-05
5    2019-01-06
6    2019-01-07
7    2019-01-08
8    2019-01-09
9    2019-01-10
Name: Datum, dtype: object
Parsed 'Datum' values:
0   2019-01-01
1   2019-01-02
2   2019-01-03
3   2019-01-04
4   2019-01-05
5   2019-01-06
6   2019-01-07
7   2019-01-08
8   2019-01-09
9   2019-01-10
Name: Datum, dtype: datetime64[ns]
Processed data:
       Datum  Lufttemperatur
0 2019-01-01           274.1
1 2019-01-02           -24.0
2 2019-01-03          -102.6
3 2019-01-04           141.1
4 2019-01-05            76.4
Hourly data saved to: smhi_data_2022-today\hourly_consolidated_parameter_2.csv


In [91]:
convert_daily_to_hourly(r"smhi_data_2022-today", "consolidated_parameter_5.csv", 1)

Raw 'Datum' values:
0    2019-01-01
1    2019-01-02
2    2019-01-03
3    2019-01-04
4    2019-01-05
5    2019-01-06
6    2019-01-07
7    2019-01-08
8    2019-01-09
9    2019-01-10
Name: Datum, dtype: object
Parsed 'Datum' values:
0   2019-01-01
1   2019-01-02
2   2019-01-03
3   2019-01-04
4   2019-01-05
5   2019-01-06
6   2019-01-07
7   2019-01-08
8   2019-01-09
9   2019-01-10
Name: Datum, dtype: datetime64[ns]
Processed data:
       Datum  Nederbördsmängd
0 2019-01-01             22.3
1 2019-01-02             22.2
2 2019-01-03             37.0
3 2019-01-04             88.3
4 2019-01-05             13.4
Hourly data saved to: smhi_data_2022-today\hourly_consolidated_parameter_5.csv


In [92]:
convert_daily_to_hourly(r"smhi_data_2022-today", "consolidated_parameter_8.csv", 1)

Raw 'Datum' values:
0    2019-01-01
1    2019-01-02
2    2019-01-03
3    2019-01-04
4    2019-01-05
5    2019-01-06
6    2019-01-07
7    2019-01-08
8    2019-01-09
9    2019-01-10
Name: Datum, dtype: object
Parsed 'Datum' values:
0   2019-01-01
1   2019-01-02
2   2019-01-03
3   2019-01-04
4   2019-01-05
5   2019-01-06
6   2019-01-07
7   2019-01-08
8   2019-01-09
9   2019-01-10
Name: Datum, dtype: datetime64[ns]
Processed data:
       Datum  Snödjup
0 2019-01-01     1.22
1 2019-01-02     1.15
2 2019-01-03     1.34
3 2019-01-04     1.28
4 2019-01-05     1.00
Hourly data saved to: smhi_data_2022-today\hourly_consolidated_parameter_8.csv


In [103]:
import os
import pandas as pd

def combine_csv_by_datum_and_tid(folder_path, output_file_name):
    
    csv_files = [f for f in os.listdir(folder_path) if f.endswith(".csv")]
    final_combined_df = pd.DataFrame()
    
    for file in csv_files:
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        df["Datum"] = pd.to_datetime(df["Datum"], errors='coerce', dayfirst=True)
        df["Tid (UTC)"] = pd.to_datetime(df["Tid (UTC)"], errors='coerce', format='%H:%M:%S')
        df["Date"] = df["Datum"].dt.strftime('%Y-%m-%d')
        df["Time"] = df["Tid (UTC)"].dt.strftime('%H:%M:%S')
        
        df = df.drop(columns=["Datum", "Tid (UTC)"])
        df = df.rename(columns={"Solskenstid": "Sunshine time"})
        
        if final_combined_df.empty:
            final_combined_df = df
        else:
            final_combined_df = pd.merge(final_combined_df, df, on=["Date", "Time"], how="outer")

    final_combined_df = final_combined_df.groupby(["Date", "Time"]).first().reset_index()

    final_combined_df = final_combined_df.sort_values(by=["Date", "Time"]).reset_index(drop=True)

    final_combined_df["Date"] = pd.to_datetime(final_combined_df["Date"], errors='coerce')

    missing_sunshine_data = final_combined_df[(final_combined_df["Sunshine time"].isna()) & 
                                              (final_combined_df["Date"] >= pd.to_datetime("2019-01-13"))]

    if not missing_sunshine_data.empty:
        print(missing_sunshine_data[['Date', 'Time']])

    final_combined_df["Sunshine time"] = final_combined_df["Sunshine time"].fillna(method='ffill')

    output_file_path = os.path.join(folder_path, output_file_name)
    final_combined_df.to_csv(output_file_path, index=False)
    
    print(f"Combined CSV saved to: {output_file_path}")


combine_csv_by_datum_and_tid(r"Hourly_data", "combined_output.csv")


Error parsing 'Datum' column in combined_output.csv: 'Datum'
            Date      Time
288   2019-01-13  00:00:00
289   2019-01-13  01:00:00
290   2019-01-13  02:00:00
291   2019-01-13  03:00:00
292   2019-01-13  04:00:00
...          ...       ...
51859 2024-11-30  19:00:00
51860 2024-11-30  20:00:00
51861 2024-11-30  21:00:00
51862 2024-11-30  22:00:00
51863 2024-11-30  23:00:00

[31673 rows x 2 columns]


C:\Users\sneha\AppData\Local\Temp\ipykernel_15444\1946782504.py:81: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  final_combined_df["Sunshine time"] = final_combined_df["Sunshine time"].fillna(method='ffill')  # Forward fill


Combined CSV saved to: Hourly_data\combined_output.csv


# Combine SHMI data with electricity price data

In [110]:
import os
import pandas as pd

def combine_electricity_data(folder_path, output_file_name):
    csv_files = [f for f in os.listdir(folder_path) if f.startswith("SE3") and f.endswith(".csv") and any(year in f for year in ['2019', '2020', '2021', '2022', '2023', '2024'])]

    df_list = []
    
    for file in csv_files:
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)

        df["Date"] = df["MTU (CET/CEST)"].str.split(' ', expand=True)[0]
        df["Time"] = df["MTU (CET/CEST)"].str.split(' ', expand=True)[1]
        
        df = df.drop(columns=["MTU (CET/CEST)"])
        
        df["Date"] = pd.to_datetime(df["Date"], format='%d.%m.%Y', errors='coerce')
        
        df_list.append(df)

    final_combined_df = pd.concat(df_list, ignore_index=True)

    final_combined_df = final_combined_df.sort_values(by=["Date", "Time"]).reset_index(drop=True)
    
    output_file_path = os.path.join(folder_path, output_file_name)
    final_combined_df.to_csv(output_file_path, index=False)
    
    print(f"Combined CSV saved to: {output_file_path}")

combine_electricity_data(r"electricity", "combined_electricity_2019_2024.csv")


Combined CSV saved to: electricity\combined_electricity_2019_2024.csv


In [113]:
import pandas as pd

def merge_electricity_with_weather(electricity_file, weather_file, output_file_name):
    electricity_df = pd.read_csv(electricity_file)
    weather_df = pd.read_csv(weather_file)

    electricity_df['Date'] = pd.to_datetime(electricity_df['Date'], errors='coerce').dt.strftime('%Y-%m-%d')
    weather_df['Date'] = pd.to_datetime(weather_df['Date'], errors='coerce').dt.strftime('%Y-%m-%d')
    
    electricity_df['Time'] = pd.to_datetime(electricity_df['Time'], errors='coerce').dt.strftime('%H:%M:%S')
    weather_df['Time'] = pd.to_datetime(weather_df['Time'], errors='coerce').dt.strftime('%H:%M:%S')
    
    electricity_df['Date'] = electricity_df['Date'].str.strip()
    electricity_df['Time'] = electricity_df['Time'].str.strip()
    weather_df['Date'] = weather_df['Date'].str.strip()
    weather_df['Time'] = weather_df['Time'].str.strip()

    electricity_df = electricity_df.rename(columns={"Day-ahead Price [EUR/MWh]": "price"})

    merged_df = pd.merge(weather_df, electricity_df[["Date", "Time", "price"]],
                         on=["Date", "Time"], how="left")
    

    merged_df = merged_df.sort_values(by=["Date", "Time"]).reset_index(drop=True)
    
    output_file_path = output_file_name
    merged_df.to_csv(output_file_path, index=False)
    
    print(f"Merged CSV saved to: {output_file_path}")

merge_electricity_with_weather(
    r"electricity/combined_electricity_2019_2024.csv", 
    r"final_combined.csv", 
    "merged_output.csv"
)


C:\Users\sneha\AppData\Local\Temp\ipykernel_15444\2586221271.py:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  electricity_df['Time'] = pd.to_datetime(electricity_df['Time'], errors='coerce').dt.strftime('%H:%M:%S')
C:\Users\sneha\AppData\Local\Temp\ipykernel_15444\2586221271.py:26: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  weather_df['Time'] = pd.to_datetime(weather_df['Time'], errors='coerce').dt.strftime('%H:%M:%S')


Merged CSV saved to: merged_output.csv


# Final data

In [2]:
import pandas as pd


df = pd.read_csv("final_combined.csv")


df = df.dropna()


df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
df = df.dropna(subset=['Price'])


df.to_csv("final_cleaned.csv", index=False)

print("Data cleaned successfully. Saved as 'final_cleaned.csv'.")


Data cleaned successfully. Saved as 'final_cleaned.csv'.
